# Milestone 4 Checkpoint

LangAlpha is a multi-agent AI equity analysis tool designed to provide comprehensive insights into the stock market. It leverages Large Language Models (LLMs) and agentic workflows to automate data gathering, processing, and analysis.

## Key Technologies

*   **Programming Language:** Python
*   **AI/LLM Frameworks:** LangChain, LangGraph
*   **Core Agent Workflow:** Implemented in `src/agent/market_intelligence_agent` using LangGraph
*   **Data Sources & Tools:**
    *   **Market/Fundamental Data:** Polygon, Yahoo Finance (via `src/mcp_server/market_data.py`, `src/mcp_server/fundamental_data.py`)
    *   **News/Web Research:** Tavily Search, Tickertick News API (via `src/mcp_server/tavily.py`, `src/mcp_server/tickertick.py`)
    *   **Web Browsing:** Playwright (integrated via `browser` agent)
    *   **Code Execution:** Local Python/Bash environment (via `coder` agent)
    *   **Database:** We will use mongoDB to store the query history and the analysis result in production phase


## Market Intelligence Agent Workflow

The primary functionality is delivered through a sophisticated **multi-agent system** built with **LangGraph**, located in `src/agent/market_intelligence_agent/`.

**Workflow Overview:**

1.  **User Query:** The system accepts natural language queries about financial markets, stocks, or economic events.
2.  **Supervisor & Planner:** A `supervisor` agent oversees the entire process. It first consults a `planner` agent, which breaks down the user's request into a detailed, step-by-step research and analysis plan.
3.  **Information Gathering (Parallel & Iterative):**
    *   The `supervisor` delegates tasks based on the plan to specialized agents:
        *   **`researcher`:** Fetches news (Tickertick) and performs general web searches (Tavily) for qualitative information, recent events, and context.
        *   **`market`:** Retrieves quantitative data like stock prices, technical indicators, fundamental data (financials, valuation), and related metrics using dedicated tools connected to Polygon and Yahoo Finance.
        *   **`browser`:** (Used sparingly due to cost) Performs deep web browsing for specific, hard-to-find information on given URLs when the `researcher`'s tools are insufficient.
4.  **Analysis & Synthesis:**
    *   **`coder`:** (If required by the plan) Executes Python code for complex calculations, data manipulation, or analysis beyond the `market` agent's built-in capabilities.
    *   **`analyst`:** Acts as a financial expert (L/S Hedge Fund perspective). It synthesizes the information gathered by all other agents, identifies key insights, assesses risks, generates potential investment theses (Long/Short), and provides actionable financial analysis based *only* on the provided data.
5.  **Reporting:**
    *   The `supervisor` reviews the findings from all agents, potentially iterating or requesting clarifications.
    *   Finally, a `reporter` agent compiles all the validated information and analysis into a comprehensive, well-structured Markdown report, including tables and summaries.

**Key Capabilities:**

*   **Autonomous Research:** Agents collaboratively gather diverse data types (news, market data, fundamentals, web content).
*   **In-depth Analysis:** Combines quantitative data retrieval with qualitative research and expert financial analysis.
*   **Structured Planning:** Ensures a logical flow of information gathering and analysis tailored to the user's query.
*   **Flexible Orchestration:** The `supervisor` dynamically routes tasks and manages the workflow, allowing for iteration and refinement.
*   **Actionable Insights:** Aims to provide not just data, but synthesized analysis suitable for investment decision-making context (though not direct financial advice).

## Damodaran Valuation Model:
- Implements valuation methodologies inspired by Professor Aswath Damodaran.
- Provides a user interface (details TBD) for manual input and valuation generation.
- See the [ginzu_interface.ipynb](https://github.com/Chen-zexi/LangAlpha/blob/master/notebooks/ginzu_interface.ipynb) for details.

## Trading Strategy:
- See the [counter-trend-trading-strategy colab](https://colab.research.google.com/drive/1Wo1f5SvZ3M9YjUx7gl4q2rSIo7PMegBN?usp=sharing#scrollTo=1kA-HcwGkK9Z) for details.



## Important Notes for Graders
- **DO NOT RUN THIS NOTEBOOK**: The source code are not accessible when submitting a single notebook
- The complete codebase can be accessed at [GitHub Repository](https://github.com/Chen-zexi/LangAlpha)

Below is an image demonstrate the current agent workflow
![graph](https://github.com/Chen-zexi/LangAlpha/blob/master/assets/graph.png)

In [1]:
import sys
import os
import warnings
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../'))
warnings.filterwarnings("ignore", category=FutureWarning)
from src.agent.market_intelligence_agent.service.workflow_service_dev import run_workflow

In [2]:
raw_response, final_report = await run_workflow(user_query="Should I invest meta now? Provide me insight for both long term and short term opportunity evaluation for meta. Make sure you include techinical indicator, fundamental outlook, and trading signal. I want you to dive deep, provide me a comprehensive but deep report. Ask your self follow up question and provide me more inisght and context")

2025-04-28 07:34:54,704 - httpx - INFO - HTTP Request: POST http://localhost:2024/threads "HTTP/1.1 200 OK"
2025-04-28 07:34:54,707 - httpx - INFO - HTTP Request: POST http://localhost:2024/threads/c44cf367-e0af-413d-a053-a48bf2e6e935/runs/stream "HTTP/1.1 200 OK"


Starting graph stream for query: 'Should I invest meta now? Provide me insight for both long term and short term opportunity evaluation for meta. Make sure you include techinical indicator, fundamental outlook, and trading signal. I want you to dive deep, provide me a comprehensive but deep report. Ask your self follow up question and provide me more inisght and context'...
------------------------------------------------------------------------------------------------------------------------
Coordinator handed off the task to the team.
------------------------------------------------------------------------------------------------------------------------
Planner is thinking...
------------------------------------------------------------------------------------------------------------------------
Comprehensive Long- & Short-Term Investment Evaluation for Meta Platforms (META)
---------------------------------------------------------------------------------------------------------------

## Formate the response with Markdown

In [3]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(final_report))

# Meta Platforms (META): Comprehensive Investment Report  
*April 28, 2025*

---

## Executive Summary

**Meta Platforms (META)** stands as a global leader in digital advertising and social platforms, with robust secular growth drivers in AI, messaging, and hardware innovation. Despite ongoing regulatory and Reality Labs headwinds, Meta’s core business generates exceptional free cash flow and trades at a discount to both intrinsic value and key peers.  
- **Long-Term Outlook:** **Buy on weakness below $525**; fair value range $700–$792 (DCF/peer-based).  
- **Short-Term Outlook:** **Neutral-bullish**; tactical long setup ($525–$530 entry, $555–$560 target).  
- **Key Catalysts:** Q1 2025 earnings (April 30), new hardware launches, regulatory/legal developments.  
- **Major Risks:** Regulatory fines, Reality Labs losses, macro ad-spend shocks.

---

## Market Context

- **Macro:** Tech stocks face mixed macro data and regulatory scrutiny, but AI and hardware innovation remain strong sector themes.
- **META Positioning:** Shares have lagged some AI peers but are consolidating after a strong multi-year run. Regulatory and content-liability risks weigh on valuation, while heavy AI and AR/VR investments create both opportunity and drag.

---

## Key Findings

- **Ad Revenue Resilience:** Core ad business remains robust, with Q1 2025 consensus +13% YoY. AI-driven ad targeting and Reels monetization are expanding ARPU.
- **AI & Product Innovation:** Heavy investment in AI (ad targeting, Llama models) and hardware (Ray-Ban Meta Glasses) positions Meta for future growth.
- **Cost Controls:** Management is curbing Reality Labs losses and overall capex, supporting margin expansion.
- **Valuation:** P/E (22.9x) and EV/EBITDA (16.2x) are below GOOGL despite similar growth/margins. DCF-based fair value is ~$792 (48% upside).
- **Regulatory/Legal:** Fines in Nigeria, lawsuits in Ghana, and content scrutiny in India are headline risks, but current costs are manageable (<1% of FCF).

---

## Detailed Analysis

### 1. Fundamental Metrics & Peer Comparison

| Metric         | META   | GOOGL  | SNAP   | PINS   | 5-Yr META Avg |
|----------------|--------|--------|--------|--------|--------------|
| **P/E**        | 22.9x  | 25.1x  | N/A    | 29.4x  | 28.7x        |
| **EV/EBITDA**  | 16.2x  | 18.0x  | N/A    | 22.1x  | 19.5x        |
| **P/S**        | 7.2x   | 7.8x   | 5.1x   | 8.2x   | 8.4x         |
| **PEG**        | 1.1x   | 1.2x   | N/A    | 1.5x   | 1.4x         |

*Meta trades at a discount to GOOGL and its own 5-year average, despite similar or better growth/margins.*

#### Key Financials (TTM)
- **Revenue:** $185.8B (+20.6% YoY)
- **Net Income:** $70.6B (+51.4% YoY)
- **Gross Margin:** 81.7%
- **Operating Margin:** 45.3%
- **Free Cash Flow Margin:** ~37%
- **Net Debt:** $49.8B; **Cash & Equivalents:** ~$40B
- **Current Ratio:** 2.98

---

### 2. Technical Analysis & Trading Signals

| Indicator         | Value      | Signal/Trend         |
|-------------------|------------|----------------------|
| **Price**         | $533       | Consolidating        |
| **20-day SMA**    | $528 (↑)   | Support zone         |
| **50-day SMA**    | $542 (↓)   | Resistance           |
| **200-day SMA**   | $496 (↑)   | Long-term support    |
| **RSI (14)**      | 53         | Neutral              |
| **MACD**          | Flat above 0 | Awaiting catalyst  |
| **Bollinger Bands** | $517–$550 | Tightening range     |

- **Support:** $520 (20-day SMA), then $500 (200-day SMA)
- **Resistance:** $550 (50-day SMA), then $580–$600 (Mar–Apr highs)
- **Pattern:** Higher-low sequence since Dec 2024; 20/50-day “death cross” in mid-April suggests short-term consolidation

---

### 3. SWOT Analysis

| Strengths                | Weaknesses                |
|--------------------------|---------------------------|
| Dominant ad platform     | Reality Labs drag         |
| AI/ML leadership         | Regulatory scrutiny       |
| Strong FCF, cash return  | High capex in AI/VR       |
| Product innovation       | Content moderation costs  |

| Opportunities            | Threats                   |
|--------------------------|---------------------------|
| AI monetization          | Macro ad-spend shocks     |
| Hardware ecosystem       | Regulatory/legal actions  |
| Messaging/commerce       | Competition (GOOGL, TikTok)|
| Cost discipline          | Tech platform risk        |

---

### 4. Scenario Analysis

| Scenario                | Impact on META            | Probability | Actionable Response         |
|-------------------------|--------------------------|-------------|----------------------------|
| **Strong Q1 Earnings**  | Breakout above $555, targets $580–$600 | Medium-High | Add to tactical long      |
| **Regulatory Escalation** | Fines, ARPU hit, sentiment drag | Medium      | Monitor, hedge exposure   |
| **Reality Labs Losses Persist** | EPS drag, but manageable | Medium      | Watch for capex guidance  |
| **Macro Ad-Spend Recession** | Revenue/EPS -6–12% | Low-Medium  | Reduce position, wait for reset |

---

### 5. Risk Factors

- **Macro/Ad-Spend Sensitivity:** 5% ad-spend recession could cut revenue by 6–8% and EPS by 10–12%. Cost controls and non-ad revenue (WhatsApp, VR) provide some cushion.
- **Reality Labs Losses:** Each $1B in extra losses reduces EPS by ~$0.40. Management aims to cut capex and achieve operating leverage by FY26.
- **Regulatory/Legal:** EU/UK privacy rules could reduce ARPU by 2–3%. Fines/compliance costs (~$500M/year) are manageable relative to >$60B annual FCF, but escalation is a risk.
- **Technical Downside:** Failure to hold $520 support could see a move to $500 (200-day SMA), representing ~6% downside from current levels.

---

## Recommendation Matrix

| Horizon         | Bias/Action         | Entry Zone      | Target(s)         | Stop-Loss      | Conviction   |
|-----------------|---------------------|-----------------|-------------------|---------------|--------------|
| **Long-Term**   | Buy/Accumulate      | <$525           | $700–$792         | $500          | High         |
| **Short-Term**  | Tactical Long       | $525–$530       | $555–$560         | $515          | Medium       |
| **Swing Trade** | Bullish on Breakout | >$555 (close + vol) | $580–$600     | $550          | Medium-High  |

---

## Conclusions & Actionable Insights

- **Long-Term Investors:** Accumulate META on weakness below $525. The company’s dominant ad platform, AI innovation, and strong cash flow support a fair value of $700–$792. Regulatory and Reality Labs risks are real but manageable.
- **Short-Term Traders:** Consider tactical long positions on dips to $525–$530, targeting $555–$560. Watch for a breakout above $555 (with volume) for a swing to $580–$600.
- **Monitor:** Q1 earnings (April 30), regulatory/legal headlines, and technical support at $520.

---

> **Disclaimer:** This report is for informational purposes only and does not constitute investment advice. Past performance is not indicative of future results. Always conduct your own due diligence and consider your risk tolerance before making any trade or investment.

---

**Sources:**  
- Company filings (10-K/10-Q), MarketBeat, Business Insider, Forbes, Digital Information World, Engadget, Africanews, Leadership.ng, Economic Times India, Gizchina, analyst synthesis.

---

**Prepared by:**  
*Professional Financial Reporter – April 28, 2025*